# Lab 3 - Extending Logistic Regression 

- Ayesh Madushanka Mahamada Kalapuwage
- Bárbara Maria Teixeira Costa Peluzo

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline


## Preparation and Overview

Let's open and visualize our data:

In [4]:
df=pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Gender                          2111 non-null   object 
 1   Age                             2111 non-null   float64
 2   Height                          2111 non-null   float64
 3   Weight                          2111 non-null   float64
 4   family_history_with_overweight  2111 non-null   object 
 5   FAVC                            2111 non-null   object 
 6   FCVC                            2111 non-null   float64
 7   NCP                             2111 non-null   float64
 8   CAEC                            2111 non-null   object 
 9   SMOKE                           2111 non-null   object 
 10  CH2O                            2111 non-null   float64
 11  SCC                             2111 non-null   object 
 12  FAF                             21

In [5]:
df

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.000000,1.620000,64.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,0.000000,1.000000,no,Public_Transportation,Normal_Weight
1,Female,21.000000,1.520000,56.000000,yes,no,3.0,3.0,Sometimes,yes,3.000000,yes,3.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.000000,1.800000,77.000000,yes,no,2.0,3.0,Sometimes,no,2.000000,no,2.000000,1.000000,Frequently,Public_Transportation,Normal_Weight
3,Male,27.000000,1.800000,87.000000,no,no,3.0,3.0,Sometimes,no,2.000000,no,2.000000,0.000000,Frequently,Walking,Overweight_Level_I
4,Male,22.000000,1.780000,89.800000,no,no,2.0,1.0,Sometimes,no,2.000000,no,0.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,20.976842,1.710730,131.408528,yes,yes,3.0,3.0,Sometimes,no,1.728139,no,1.676269,0.906247,Sometimes,Public_Transportation,Obesity_Type_III
2107,Female,21.982942,1.748584,133.742943,yes,yes,3.0,3.0,Sometimes,no,2.005130,no,1.341390,0.599270,Sometimes,Public_Transportation,Obesity_Type_III
2108,Female,22.524036,1.752206,133.689352,yes,yes,3.0,3.0,Sometimes,no,2.054193,no,1.414209,0.646288,Sometimes,Public_Transportation,Obesity_Type_III
2109,Female,24.361936,1.739450,133.346641,yes,yes,3.0,3.0,Sometimes,no,2.852339,no,1.139107,0.586035,Sometimes,Public_Transportation,Obesity_Type_III


<b>Question:</b> Explain the task and what business-case or use-case it is designed to solve (or designed to investigate). Detail exactly what the classification task is and what parties would be interested in the results. For example, would the model be deployed or used mostly for offline analysis? 

<b>Question:</b> Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc. Remove variables that are not needed/useful for the analysis. Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created).

### Pre-processing:

As it was explainedm in the first lab, there are a number of categories, which need to have their instances converted to integers (both binary or frequency type). Moreover, it is necessary to turn some of the floats to integers (features which entries should be integers, but some floats are displayed) and remove the <b> smoking </b> atribute. 

With respect to missing or duplicated data, it is not necessary to input data and we will not deleted the few duplicated entries. Finally, we will also change the atributes names, in order to simplify them.

In [28]:
df.rename(columns = {'family_history_with_overweight': 'family', 'FAVC': 'caloric_food', 'FCVC': 'vegetables', 'NCP': 'n_meal', 'CAEC':'eat_bet_meal', 'SMOKE': 'smoking', 'CH2O' : 'water', 'SCC' : 'monitor_cal', 'FAF' : 'physical_active', 'TUE': 'screen_time', 'CALC': 'alcohol', 'MTRANS': 'transport', 'NObeyesdad': 'obesity'}, inplace=True)
df.Age=round(df.Age).astype(int)
df.water=round(df.water).astype(int)
df.physical_active=round(df.physical_active).astype(int)
df.screen_time=round(df.screen_time).astype(int)
df.vegetables=df.vegetables.astype(int)
df.n_meal=df.n_meal.astype(int)
del df['smoking']

In [30]:
frequency=['no','Sometimes','Frequently','Always']
interval=range(4)
yn=['no','yes']
binary=range(2)
df.family.replace(to_replace=yn[:],
           value=binary,
           inplace=True)
df.caloric_food.replace(to_replace=yn[:],
                        value=binary,
                        inplace=True)
df.monitor_cal.replace(to_replace=yn[:],
                       value=binary,
                       inplace=True)
df.eat_bet_meal.replace(to_replace=frequency[:],
                        value=interval,
                        inplace=True)
df.alcohol.replace(to_replace=frequency[:],
                   value=interval,
                   inplace=True)
df.transport.replace(to_replace=['Walking','Bike','Public_Transportation','Motorbike','Automobile'],
                     value=range(5),
                     inplace=True)
df.vegetables.replace=df.vegetables[:]-1

df

,Gender,Age,Height,Weight,family,caloric_food,vegetables,n_meal,eat_bet_meal,water,monitor_cal,physical_active,screen_time,alcohol,transport,obesity
0,Female,21,1.620000,64.000000,1,0,2,3,1,2,0,0,1,0,2,Normal_Weight
1,Female,21,1.520000,56.000000,1,0,3,3,1,3,1,3,0,1,2,Normal_Weight
2,Male,23,1.800000,77.000000,1,0,2,3,1,2,0,2,1,2,2,Normal_Weight
3,Male,27,1.800000,87.000000,0,0,3,3,1,2,0,2,0,2,0,Overweight_Level_I
4,Male,22,1.780000,89.800000,0,0,2,1,1,2,0,0,0,1,2,Overweight_Level_II
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,Female,21,1.710730,131.408528,1,1,3,3,1,2,0,2,1,1,2,Obesity_Type_III
2107,Female,22,1.748584,133.742943,1,1,3,3,1,2,0,1,1,1,2,Obesity_Type_III
2108,Female,23,1.752206,133.689352,1,1,3,3,1,2,0,1,1,1,2,Obesity_Type_III
2109,Female,24,1.739450,133.346641,1,1,3,3,1,3,0,1,1,1,2,Obesity_Type_III


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           2111 non-null   object 
 1   Age              2111 non-null   int64  
 2   Height           2111 non-null   float64
 3   Weight           2111 non-null   float64
 4   family           2111 non-null   int64  
 5   caloric_food     2111 non-null   int64  
 6   vegetables       2111 non-null   int64  
 7   n_meal           2111 non-null   int64  
 8   eat_bet_meal     2111 non-null   int64  
 9   water            2111 non-null   int64  
 10  monitor_cal      2111 non-null   int64  
 11  physical_active  2111 non-null   int64  
 12  screen_time      2111 non-null   int64  
 13  alcohol          2111 non-null   int64  
 14  transport        2111 non-null   int64  
 15  obesity          2111 non-null   object 
dtypes: float64(2), int64(12), object(2)
memory usage: 264.0+ KB


## Modeling

## Deployment

## Exceptional Work